# Live ratio

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
from scipy import signal
from PIL import Image
from time import sleep
import picamera
import picamera.array

In [2]:
def fun_ratio(autocon):
    [sy,sx]=autocon.shape
    max_=np.amax(autocon)
    [max_y,max_x]=np.where(autocon==max_)
    max_y=float(max_y)
    max_x=float(max_x)
    disk_mask = np.ones((sy, sx), dtype=float)
    rr = np.array([max_y-1, max_y-1, max_y-1, max_y, max_y, max_y, max_y+1, max_y+1, max_y+1],dtype=int)
    cc = np.array([max_x-1, max_x, max_x+1, max_x-1, max_x, max_x+1, max_x-1, max_x, max_x+1],dtype=int)
    disk_mask[rr, cc] = 0
    autocon = np.maximum(autocon,np.ones(autocon.shape)) ### HERE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    autocon_masked=disk_mask*autocon
    sum_nz=sum(sum(autocon_masked))
    number_nz=autocon.size-len(rr)
    average_nz=sum_nz/number_nz
    ratio_nz=max_/average_nz
    return ratio_nz

Use this code to find the perfect distance live

In [3]:
if __name__== '__main__':
    camera = picamera.PiCamera()
    camera.color_effects = (128, 128) # The reason why grey image
    camera.start_preview(resolution=(410,313),fullscreen=False,window=(10,10,410,308))
    camera.exposure_mode = 'auto'
    exit = 'init'
    while exit != 'exit':
        sleep(2)
        camera.capture('from_camera/live/live_img.png', 'png')
        image_im = Image.open('from_camera/live/live_img.png')
        image_im = np.array(image_im,dtype=float)
        image_im = image_im[:, :, 0]
        image_im = image_im - image_im.mean()  #Subtract mean
        auto_im=signal.fftconvolve(image_im,np.flip(image_im),mode='same') #Autocorrelation
        rati_im=fun_ratio(auto_im)
        print(rati_im)
        exit = input('Press enter to compute the ratio. Write exit to exit :)')
    camera.stop_preview()
    print('stop')

174.7946975423862
173.05354562816194
120.04655946198447
118.76117501503339
91.90209220402983
91.16658583209748
74.39585275161771
74.61869877596153
stop


In [4]:
camera.stop_preview()